<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Kentucky_Data_API_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 14.7 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11122 sha256=917d82189e7e886b7b1f10d9657a35b14d3ff5b51af2303de544db6650649e0a
  Stored in directory: /root/.ca

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135741 sha256=54417f8f18a066d91e2ef332e48b6637ee31f69d281dfe2fb5b6ffbec76f85ee
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236757 sha256=038111aa7eb61a0770683ceabef4610fdad5c51f0796131e4d7a9a2ab9f6997d
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=70_AYEOMY0CEiAoy3tTGWlzlA4-7eDo7VTh2dv-R7vI&tc=3Q1NJXR5WbWIYluJHsXWmABvqgtqnBkwPPZH7AYBchE&cc=pBvZ3QiwkB52xXIQfbMs0Nop79unhqLtz7ducE7nmuY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWj2WzmsUn81Tgr4aXYl1M_61mzfWLU0K7yPGKRl-LyZWExF9aiCls

Successfully saved authorization token.


In [7]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/KY_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))

Data dimensions: (12606, 19)


In [8]:
# Get the shape geometry
region = eec.gdfToFc(mines)

In [9]:
mines_exploded= mines.explode()
mines_exploded

<ipython-input-9-86b5dc24b419>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded= mines.explode()


id  ACT_INAC  Calc_Acres  \
0     0  00000000000000001111  Released  294.302954   
1     0  00000000000000001112  Released  305.739886   
      1  00000000000000001112  Released  305.739886   
      2  00000000000000001112  Released  305.739886   
      3  00000000000000001112  Released  305.739886   
...                       ...       ...         ...   
12603 2  00000000000000000472  Released   56.660359   
12604 0  0000000000000000308a  Released   14.972560   
      1  0000000000000000308a  Released   14.972560   
      2  0000000000000000308a  Released   14.972560   
12605 0  000000000000000004fc  Released   35.631015   

                             Contact       DATE_ISS FeatCLS MINE_STATU  \
0     0  KY Division of Mine Permits  1141718400000      SF         FF   
1     0  KY Division of Mine Permits  1141718400000      SF         FF   
      1  KY Division of Mine Permits  1141718400000      SF         FF   
      2  KY Division of Mine Permits  1141718400000      SF         FF   
      3  KY Division of Mine Permits  1141718400000      SF         FF   
...                              ...            ...     ...        ...   
12603 2  KY Division of Mine Permits   255254400000      SF         XX   
12604 0  KY Division of Mine Permits   166863600000      SF         XX   
      1  KY Division of Mine Permits   166863600000      SF         XX   
      2  KY Division of Mine Permits   166863600000      SF         XX   
12605 0  KY Division of Mine Permits   262598400000      SF         XX   

        National_I ORIG_PERM  \
0     0  KY8640180   8640180   
1     0  KY8640180   8640180   
      1  KY8640180   8640180   
      2  KY8640180   8640180   
      3  KY8640180   8640180   
...            ...       ...   
12603 2  KY686977X   686977X   
12604 0  KY388574X   388574X   
      1  KY388574X   388574X   
      2  KY388574X   388574X   
12605 0  KY710477X   710477X   

                                                  PERM_ACT  \
0     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
1     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      1  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      2  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      3  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
...                                                    ...   
12603 2                                 NW      02/02/1978   
12604 0             AM      07/14/1975  NW      04/16/1975   
      1             AM      07/14/1975  NW      04/16/1975   
      2             AM      07/14/1975  NW      04/16/1975   
12605 0                                 NW      04/28/1978   

                        PER_NAME   PER_TYPE PermitNo       QUAD_DESC  \
0     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
1     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      1         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      2         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      3         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
...                          ...        ...      ...             ...   
12603 2         CAL GLO COAL INC    PRE LAW  686977X    BARBOURVILLE   
12604 0   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      1   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      2   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
12605 0  T & T DARBY COAL CO INC    PRE LAW  710477X  PENNINGTON GAP   

           REGION_DES    Shape_Leng       Source Type_Flag  \
0     0  PRESTONSBURG  53267.493940         CHIA      TRNS   
1     0  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      1  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      2  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      3  PRESTONSBURG  92560.008796        KYDMP      TRNS   
...               ...           ...          ...       ...   
12603 2   MIDDLESBORO  16743.038634  KGS Project

In [10]:
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')

probability_bands = [
  'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
  'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
]
palette = [
  '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635',
  '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1'
]

start_date = '2019-04-01'
end_date = '2019-07-01'

# Filter image collections by time
dw_time_interval = dw.filter(ee.Filter.date(start_date, end_date))

# Select probability bands
dw_time_series = dw_time_interval.select(probability_bands)

# Create a multi-band image summarizing probability
# for each band across the time-period
mean_probability = dw_time_series.reduce(ee.Reducer.mean())

# Create a single band image containing the class with the top probability
top_probability = mean_probability.toArray().arrayArgmax().arrayGet(0).rename('label')

geometry = ee.Geometry.Polygon(
        [[[-77.26456078565367, 38.98239629898283],
          [-77.26456078565367, 38.24414499918236],
          [-76.27579125440367, 38.24414499918236],
          [-76.27579125440367, 38.98239629898283]]])

url = top_probability.getThumbURL({
  'region': geometry,
  'dimensions':'300',
  'min': 0,
  'max': 8,
  'palette': palette,
  'format': 'png'
})

from IPython.display import Image

Image(url = url)

In [11]:
mines['id'][2]

'00000000000000001f4d'

In [12]:
region

/usr/local/lib/python3.10/dist-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Request payload size exceeds the limit: 10485760 bytes.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [ ]:
df = pd.DataFrame()

i=2
# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-31'

df_contructor = []
while i<len(mines_exploded):

        fil_mines = mines_exploded.iloc[[i]]
        region = eec.gdfToFc(fil_mines)
        # Get dynamic world image collection by asset ID
        dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)

        # get band names, equal to column names
        bandnames = dc.first().bandNames().getInfo()[:-1]

        polygon = mines_exploded['geometry'].values[i]

        if polygon.geom_type == 'Polygon':
          # Extract the coordinates from the polygon
          coordinates = polygon.exterior.coords[:]

          # Create an ee.Geometry object from the coordinates
          polygon = ee.Geometry.MultiPolygon(coordinates)

          # Set the maxError parameter
          max_error = 1  # Specify your desired maxError value

          # Project the geometry coordinates
          projected_polygon = polygon.transform('EPSG:4326', max_error)

        elif polygon.geom_type == 'LineString':
          coordinates = polygon.coords[:]
          polygon = ee.Geometry.LineString(coordinates)

        #decrease scale to get more rows of data/granularity
        ac = dc.getRegion(projected_polygon, scale=100 ).getInfo()
        df_constructor =pd.DataFrame(ac)

        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Company'] = mines_exploded['PER_NAME'].values[i]
        df = df.append(df_constructor)

        print(mines_exploded['PER_NAME'].values[i])
        i+=1


<ipython-input-14-a79f51df1927>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


B & C ENERGY INC


<ipython-input-14-a79f51df1927>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


B & C ENERGY INC


<ipython-input-14-a79f51df1927>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


...

SUNFLOWER COAL CO INC


<ipython-input-14-a79f51df1927>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


SUNFLOWER COAL CO INC


<ipython-input-14-a79f51df1927>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


FEDERAL MINING CO INC


In [ ]:
df

In [ ]:
df['Company'].unique()

In [ ]:
df.to_csv('ky_mines_jan_2021.csv', encoding = 'utf-8-sig')
files.download('ky_mines_jan_2021.csv')

In [ ]:
#https://stackoverflow.com/questions/55659835/trying-to-separate-polygon-data-into-x-and-y-coordinates-but-get-error-multip

import geopandas as gpd
from shapely.geometry import Polygon, LineString, MultiLineString

def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
         all_xy = []

In [ ]:
from functools import singledispatch
from itertools import chain
from typing import (List,
                    Tuple,
                    TypeVar)

from shapely.geometry import (GeometryCollection,
                              LinearRing,
                              LineString,
                              Point,
                              Polygon)
from shapely.geometry.base import (BaseGeometry,
                                   BaseMultipartGeometry)

Geometry = TypeVar('Geometry', bound=BaseGeometry)


@singledispatch
def to_coords(geometry: Geometry) -> List[Tuple[float, float]]:
    """Returns a list of unique vertices of a given geometry object."""
    raise NotImplementedError(f"Unsupported Geometry {type(geometry)}")


@to_coords.register
def _(geometry: Point):
    return [(geometry.x, geometry.y)]


@to_coords.register
def _(geometry: LineString):
    return list(geometry.coords)


@to_coords.register
def _(geometry: LinearRing):
    return list(geometry.coords[:-1])


@to_coords.register
def _(geometry: BaseMultipartGeometry):
    return list(set(chain.from_iterable(map(to_coords, geometry))))


@to_coords.register
def _(geometry: Polygon):
    return to_coords(GeometryCollection([geometry.exterior, *geometry.interiors]))

In [ ]:
import shapely as sh

def get_coords_from_polygon(shape):
    coords = set()

    if isinstance(shape, sh.geometry.Polygon):
        coords.update(shape.exterior.coords[:-1])
        for linearring in shape.interiors:
            coords.update(linearring.coords[:-1])
    elif isinstance(shape, sh.geometry.MultiPolygon):
        for polygon in shape:
            coords.update(get_coords_from_polygon(polygon))

    return coords

In [ ]:
# prompt: get coordinates fro mines_exploded['geometry'].values[i] where it is multipolygon

def get_coords_from_polygon(shape):
    coords = set()

    if isinstance(shape, sh.geometry.Polygon):
        coords.update(shape.exterior.coords[:-1])
        for linearring in shape.interiors:
            coords.update(linearring.coords[:-1])
    elif isinstance(shape, sh.geometry.MultiPolygon):
        for polygon in shape:
            coords.update(get_coords_from_polygon(polygon))

    return coords

mines_geometry = mines_exploded['geometry'].values[i]
coords = get_coords_from_polygon(mines_geometry)


In [ ]:
coordinates

In [ ]:
df

In [ ]:
from google.colab import files
df.to_csv('Kentucky_mines.csv', encoding = 'utf-8-sig')
files.download('Kentucky_mines.csv')